In [1]:
!pip install pyvis==0.3.1
!pip install pyaml

In [27]:
import yaml

with open('./data/test.yaml') as file:
    loaded = yaml.load(file, Loader=yaml.FullLoader)

In [28]:
for item in loaded.items():
    print(item)
# loaded.keys()

('receipt', 'Oz-Ware Purchase Invoice')
('date', datetime.date(2012, 8, 6))
('customer', {'first_name': 'Dorothy', 'family_name': 'Gale'})
('items', [{'part_no': 'A4786', 'descrip': 'Water Bucket (Filled)', 'price': 1.47, 'quantity': 4}, {'part_no': 'E1628', 'descrip': 'High Heeled "Ruby" Slippers', 'size': 8, 'price': 133.7, 'quantity': 1}])
('items_0', [['items_2'], ['a', 'b', 'c']])
('bill-to', {'street': '123 Tornado Alley\nSuite 16\n', 'city': 'East Centerville', 'state': 'KS'})
('ship-to', {'street': '123 Tornado Alley\nSuite 16\n', 'city': 'East Centerville', 'state': 'KS'})
('specialDelivery', 'Follow the Yellow Brick Road to the Emerald City. Pay no attention to the man behind the curtain.')


In [39]:
from typing import List, Dict, Any
from itertools import count
from dataclasses import dataclass, field
from pyvis.network import Network as PyvisNetwork
import random
import os
import shutil
import yaml


class Default:
    def print(self):
        fields = [(attribute, value) for attribute, value in self.__dict__.items()]
        print('-'*30, f'{self.__class__.__name__}', '-'*30)
        for field in self.__dict__.items():
            test = field[0]
            print(f"{field[0]}: {field[1]}")

@dataclass()
class Node(Default):
    id: int = field(default_factory=lambda count_=count(): next(count_), init=False)
    title: str = field(default_factory=str)
    description: str = field(default_factory=str)
    relations: List[int] = field(default_factory=list[int])
    # relations: List['Node'] = field(default_factory=list)
    metadata: Dict[str, Any] = field(default_factory=dict)

@dataclass()
class Edge(Default):
    id: int = field(default_factory=lambda count_=count(): next(count_), init=False)
    title: str = field(default_factory=str)
    description: str = field(default_factory=str)
    tail: 'Node' = field(default_factory='Node')
    head: 'Node' = field(default_factory='Node')
    # relations: List['Edge'] = field(default_factory=list)
    metadata: Dict[str, Any] = field(default_factory=dict)
    
    def __post_init__(self):
        self.tail.relations.append(self.head.id)
        self.head.relations.append(self.tail.id)

@dataclass()
class Graph(Default):
    nodes: List['Node'] = field(default_factory=list['Node'])
    edges: List['Edge'] = field(default_factory=list['Edge'])

    def from_yaml(self, file_path: str = './data/graphs/test_0.yaml'):
        with open(file_path) as file:
            loaded = yaml.load(file, Loader=yaml.FullLoader)
            for node in loaded['NODES']:
                self.nodes.append(Node(title=node))
            for edge in loaded['EDGES']:
                self.edges.append(Edge(head=self.get_node(edge.get('head')), tail=self.get_node(edge.get('tail')), title=edge.get('title')))

    def get_node(self, title: str):
        for node in self.nodes:
            if node.title == title:
                return node
        return None
    def save_to_html(self, filename="data/network.html", proxied=True, remove_lib=False, directed=False):
        if filename.startswith('./'):
            filename = filename[2:]
        if filename.startswith('/'):
            filename = filename[1:]
        # net = PyvisNetwork(directed=directed, width="1920px", height="1080px", bgcolor="#222222")
        net = PyvisNetwork(directed=directed, width="1920px", height="1080px", bgcolor="#eeeeee")
        proxy_address = "https://vscode.kripso-world.com/proxy/5501"

        for node in self.nodes:
            hexadecimal = ["#"+''.join([random.choice('ABCDEF0123456789') for i in range(6)])][0]
            net.add_node(node.id, label=node.title, shape="circle", color=hexadecimal)

        for edge in self.edges:
            net.add_edge(edge.head.id, edge.tail.id, title=edge.title, label=edge.title)
        net.force_atlas_2based()
        net.repulsion(
            node_distance=200,
            central_gravity=0.2,
            spring_length=100,
            spring_strength=0.05,
            damping=0.09
        )

        os.makedirs('./data', exist_ok=True)

        net.set_edge_smooth('dynamic')
        net.show(f"./{filename}")
        
        if remove_lib and os.path.isdir('./lib'):
            shutil.rmtree('./lib')
        
        if proxied:
            print(f"{proxy_address}/{filename}")
        else:
            print(f"./{filename}")

In [35]:
nodes = [Node(title=f'test_{i}') for i in range(10)]
edges = [Edge(title=f'test_{i}', tail=random.choice(nodes), head=random.choice(nodes)) for i in range(20)]

graph = Graph(
    nodes=nodes,
    edges=edges,
)

graph.save_to_html(filename='./data/network.html', directed=True, remove_lib=True)

https://vscode.kripso-world.com/proxy/5501/data/network.html


In [45]:
new_graph = Graph()
new_graph.from_yaml()
new_graph.save_to_html(filename='./data/network.html', directed=True, remove_lib=True)

https://vscode.kripso-world.com/proxy/5501/data/network.html
